In [1]:
import ast, copy, dis
import nbimport, sublambda

In [2]:
import Episode12

# Step 1: Write (or copy) a hygiene test case

In [3]:
test_code = '''
def wonkification():
    z = 42
    f(99)
    return z
'''
test_substitution = ('f', 'lambda x: (z := x ** 2, z + x + 1)[-1]')

In [5]:
f = lambda x: (z := x ** 2, z + x + 1)[-1]

def wonkification():
    z = 42
    f(99)
    return z

wonkification()

42

In [8]:
def test_transform(build_transformer):
    transformer = build_transformer(*test_substitution)
    transformed = transformer(test_code)
    exec(compile(transformed, '<string>', 'exec'), globals())
    return wonkification() == 42

test_transform(Episode12.inline_lambda)

Name(id='x', ctx=Load())
-->
Constant(value=99)
Name(id='x', ctx=Load())
-->
Constant(value=99)
Call(
  func=Name(id='f', ctx=Load()),
  args=[
    Constant(value=99)],
  keywords=[])
-->
Subscript(
  value=Tuple(
    elts=[
      NamedExpr(
        target=Name(id='z', ctx=Store()),
        value=BinOp(
          left=Constant(value=99),
          op=Pow(),
          right=Constant(value=2))),
      BinOp(
        left=BinOp(
          left=Name(id='z', ctx=Load()),
          op=Add(),
          right=Constant(value=99)),
        op=Add(),
        right=Constant(value=1))],
    ctx=Load()),
  slice=UnaryOp(
    op=USub(),
    operand=Constant(value=1)),
  ctx=Load())
===


False

In [9]:
wonkification()

9801

# Step 2: Watch the `sublambda` little language fail...

Hint: `%%mycell sublambda.sublambdas('transformer_name')`

In [13]:
%%mycell sublambda.sublambdas('mytransformer')
mypoly lambda x: x ** 2 + x + 1

<function sublambda.inline_lambdas.<locals>.inliner(source)>

In [14]:
%%mycell mytransformer
mypoly(42)

In [45]:
from importlib import reload
reload(sublambda)

<module 'sublambda' from '/Users/jon/git/ohop/ohop/sublambda.py'>

In [46]:
%%mycell sublambda.sublambdas('mytransformer')
mypoly lambda x: x ** 2 + x + 1

<function sublambda.inline_lambdas.<locals>.inliner(source)>

In [47]:
%%mycell mytransformer
foo = mypoly(42)

In [48]:
foo

1807

# Step 3: Solve for our test case